In [23]:
import tensorflow as tf
from typing import List, Tuple

In [25]:
import sys
sys.path.append('../../../../')

In [2]:
tf.config.set_visible_devices([], 'GPU')

2025-04-06 21:56:16.888132: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2025-04-06 21:56:16.889441: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2025-04-06 21:56:16.891218: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2025-04-06 21:56:16.892819: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2027] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, whi

In [3]:
from transformer_encoder_mlp import TransformerEncoderMLP

In [17]:
# Example input
inputs = tf.keras.Input(shape=(11,))
x = tf.keras.layers.Dense(1024, activation='relu')(inputs)
x = TransformerEncoderMLP(hidden_units=(1024, 1024), dropout_rate=0.2)(x)
model = tf.keras.Model(inputs, x)
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 11)]              0         
                                                                 
 dense_16 (Dense)            (None, 1024)              12288     
                                                                 
 transformer_encoder_mlp_7 (  (None, 1024)             2101248   
 TransformerEncoderMLP)                                          
                                                                 
Total params: 2,113,536
Trainable params: 2,113,536
Non-trainable params: 0
_________________________________________________________________


In [21]:
model(tf.random.uniform((10, 11)))

<tf.Tensor: shape=(10, 1024), dtype=float32, numpy=
array([[-0.6960208 , -1.5768466 ,  0.07993226, ..., -0.14368364,
         2.5021944 , -0.41295868],
       [-0.13876311, -1.4246821 ,  0.41824263, ...,  0.49558175,
         3.446252  ,  0.28421518],
       [-1.5456738 , -1.8446131 ,  0.3026702 , ..., -0.43714157,
         2.3914535 , -0.7879347 ],
       ...,
       [-0.6925067 , -1.2031287 , -0.5697024 , ...,  0.6399677 ,
         0.55358505,  0.70341927],
       [-0.7889751 , -1.3827075 , -0.33191642, ...,  0.24981944,
         1.8018522 , -0.11736801],
       [-0.41686153, -1.5389943 , -0.41158846, ...,  0.24025422,
         2.9066348 ,  0.1695547 ]], dtype=float32)>

In [98]:
from typing import Tuple
import tensorflow as tf
from models.patch_to_score.models.pts_without_mlp_b.utils import GlobalSumPooling


def broadcast_shape(x, max_number_of_patches: int) -> tf.Tensor:
    x_expanded = tf.expand_dims(x, axis=1)
    x_broadcasted = tf.broadcast_to(x_expanded, [tf.shape(x)[0], max_number_of_patches, tf.shape(x)[-1]])
    return x_broadcasted


def build_model_prev(m_a: int, m_c: int, n_layers: int, input_shape: Tuple[int, int],
                max_number_of_patches: int) -> tf.keras.models.Model:
    '''
    :param m_a: size of the hidden layers in the MLP of the components
    :param m_c: size of the hidden layers in the MLP of the concatenated global sum output and size + n_patches MLP output
    :param n_layers: number of layers in each of the MLPs
    :param input_shape: shape of the input data (number of patches, number of features)
    :param max_number_of_patches: maximum number of patches
    :return: a Keras model
    '''
    input_data = tf.keras.Input(shape=input_shape, name='patches_input')
    size_value = tf.keras.Input(shape=(1,), name='extra_value_input')
    n_patches_hot_encoded_value = tf.keras.Input(
        shape=(max_number_of_patches + 1,), name='hot_encoded_value_input')
    n_patches = tf.argmax(n_patches_hot_encoded_value, axis=1)[..., None]
    n_patches = tf.cast(n_patches, tf.float32)
    
    n_patches_broadcased = broadcast_shape(n_patches, max_number_of_patches)
    size_broadcased = broadcast_shape(size_value, max_number_of_patches)
    
    concat_input_data = tf.keras.layers.Concatenate(axis=-1)([input_data, n_patches_broadcased, size_broadcased])
    masked_input = tf.keras.layers.Masking(mask_value=0.0)(concat_input_data)

    currentOutput = masked_input
    for i in range(n_layers):
        dense_output = tf.keras.layers.Dense(
            m_a, activation='linear')(currentOutput)
        batchNorm = tf.keras.layers.BatchNormalization(
            momentum=0.75)(dense_output)
        activation = tf.keras.layers.ReLU()(batchNorm)
        currentOutput = activation


    print('before global pooling shape:', currentOutput.shape)
    global_pooling_output = GlobalSumPooling(
        data_format='channels_last')(currentOutput)
    print('after global pooling shape:', global_pooling_output.shape)

    currentOutput = global_pooling_output
    for i in range(n_layers):
        dense_output = tf.keras.layers.Dense(
            m_c, activation='linear')(currentOutput)
        batchNorm = tf.keras.layers.BatchNormalization(
            momentum=0.75)(dense_output)
        activation = tf.keras.layers.ReLU()(batchNorm)
        currentOutput = activation

    before_sigmoid_output = currentOutput

    output = tf.keras.layers.Dense(
        1, activation='sigmoid')(before_sigmoid_output)
    model = tf.keras.Model(
        inputs=[input_data, size_value, n_patches_hot_encoded_value], outputs=output)
    return model


In [107]:
from typing import Tuple, List
import tensorflow as tf
# from models.patch_to_score.models.pts_encoder_mlps.utils import GlobalSumPooling
from models.patch_to_score.models.pts_encoder_mlps.transformer_encoder_mlp import TransformerEncoderMLP


def broadcast_shape(x, max_number_of_patches: int) -> tf.Tensor:
    x_expanded = tf.expand_dims(x, axis=1)
    x_broadcasted = tf.broadcast_to(x_expanded, [tf.shape(x)[0], max_number_of_patches, tf.shape(x)[-1]])
    return x_broadcasted


def build_model(hidden_sizes_mlp_a: List[Tuple[int, int]], mlp_a_dropout_rate: float,
                hidden_sizes_mlp_c: List[Tuple[int, int]], mlp_c_dropout_rate: float,
                activation: str,
                input_shape: Tuple[int, int],
                max_number_of_patches: int) -> tf.keras.models.Model:
    '''
    :param m_a: size of the hidden layers in the MLP of the components
    :param m_c: size of the hidden layers in the MLP of the concatenated global sum output and size + n_patches MLP output
    :param n_layers: number of layers in each of the MLPs
    :param input_shape: shape of the input data (number of patches, number of features)
    :param max_number_of_patches: maximum number of patches
    :return: a Keras model
    '''
    input_data = tf.keras.Input(shape=input_shape, name='patches_input')
    size_value = tf.keras.Input(shape=(1,), name='extra_value_input')
    n_patches_hot_encoded_value = tf.keras.Input(
        shape=(max_number_of_patches + 1,), name='hot_encoded_value_input')
    n_patches = tf.argmax(n_patches_hot_encoded_value, axis=1)[..., None]
    n_patches = tf.cast(n_patches, tf.float32)
    
    n_patches_broadcased = broadcast_shape(n_patches, max_number_of_patches)
    size_broadcased = broadcast_shape(size_value, max_number_of_patches)
    
    concat_input_data = tf.keras.layers.Concatenate(axis=-1)([input_data, n_patches_broadcased, size_broadcased])
    masked_input = tf.keras.layers.Masking(mask_value=0.0)(concat_input_data)
    
    current_output = masked_input
    print(current_output.shape)
    current_output = tf.keras.layers.Dense(
        hidden_sizes_mlp_a[0][0], activation='linear')(current_output)
    
    for mlp_hidden_size in hidden_sizes_mlp_a:
        mlp = TransformerEncoderMLP(
            hidden_units=mlp_hidden_size, dropout_rate=mlp_a_dropout_rate, activation=activation)
        current_output = mlp(current_output)
        activation = tf.keras.layers.ReLU()(current_output)
        current_output = activation
        # TODO: apply non-linearity here?
    
    print('before global pool shape', current_output.shape)
    
    global_pooling_output = GlobalSumPooling(
        data_format='channels_last')(current_output)

    print('after global pooling shape', global_pooling_output.shape)
    current_output = global_pooling_output
    current_output = tf.keras.layers.Dense(
        hidden_sizes_mlp_c[0][0], activation='linear')(current_output)
    
    print('here')
    for mlp_hidden_size in hidden_sizes_mlp_c:
        mlp = TransformerEncoderMLP(
            hidden_units=mlp_hidden_size, dropout_rate=mlp_c_dropout_rate, activation=activation)
        current_output = mlp(current_output)
        # TODO: apply non-linearity here?

    before_sigmoid_output = current_output

    output = tf.keras.layers.Dense(
        1, activation='sigmoid')(before_sigmoid_output)
    model = tf.keras.Model(
        inputs=[input_data, size_value, n_patches_hot_encoded_value], outputs=output)
    return model


In [108]:
def bootstrap_pts_encoder_mlps(hidden_sizes_mlp_a: List[Tuple[int, int]], mlp_a_dropout_rate: float,
                               hidden_sizes_mlp_c: List[Tuple[int, int]], mlp_c_dropout_rate: float,
                               activation: str, input_shape: Tuple[int, int],
                               max_number_of_patches: int) -> tf.keras.models.Model:
    model = build_model(hidden_sizes_mlp_a, mlp_a_dropout_rate,
                                         hidden_sizes_mlp_c, mlp_c_dropout_rate,
                                         activation, input_shape, max_number_of_patches)
    return model

In [109]:
build_model_prev(1024, 1024, 2, [10, 9], 10)

before global pooling shape: (None, 10, 1024)
after global pooling shape: (None, 1024)


In [110]:
bootstrap_pts_encoder_mlps(
    hidden_sizes_mlp_a=[(1024, 1024), (1024, 1024)],
    mlp_a_dropout_rate=0.2,
    hidden_sizes_mlp_c=[(1024, 1024), (1024, 1024)],
    mlp_c_dropout_rate=0.2,
    activation='relu',
    input_shape=[10, 9],
    max_number_of_patches=10
)

(None, 10, 11)


TypeError: Exception encountered when calling layer "tf.keras.activations.get_1" (type TFOpLambda).

Could not interpret activation function identifier: Tensor("Placeholder:0", shape=(None, 10, 1024), dtype=float32)

Call arguments received by layer "tf.keras.activations.get_1" (type TFOpLambda):
  • identifier=tf.Tensor(shape=(None, 10, 1024), dtype=float32)